In [53]:
from lnn import Model, Predicate, Variable, And, Or, Predicates, Fact, Loss, Propositions, Implies, World, Variables
import torch.nn as nn
import gym
import numpy as np
import torch
import collections
import math

Observation = collections.namedtuple("Observation", ("Position", "Velocity", "Angle", "AngVelocity"))
Transition = collections.namedtuple("Transition", ("state", "action", "next_state", "reward", "done"))
class LNNCartpole():
    def __init__(self, num_nodes, n_pos, n_vel, n_ang, n_angvel):
        def create_predicates(n_nodes, name):
            predicate_list = []
            for i in range(n_nodes):
                predicate_list.append(Predicate(name + str(i+1)))
                predicate_list.append(Predicate(name + str(-(i+1))))
            return predicate_list

        def create_n_ary_and(num_nodes, preds):
            and_list = []
            for _ in range(num_nodes):
                and_list.append(And(*preds["Position"], *preds["Velocity"], *preds["Angle"], *preds["AngVelocity"]))
            return and_list

        def create_n_ary_or(and_list):
            return Or(*and_list)

        self.model = Model()
        self.preds = {
          "Position": create_predicates(n_pos, "pos"),
          "Velocity": create_predicates(n_vel, "vel"),
          "Angle": create_predicates(n_ang, "ang"),
          "AngVelocity": create_predicates(n_angvel, "angvel")
        }
        self.and_nodes = create_n_ary_and(num_nodes, self.preds)
        self.or_node = create_n_ary_or(self.and_nodes)

        self.model.add_knowledge(*self.and_nodes, self.or_node)
    def generate_initial_state_dictionary(self, raw_bin_dict):
        d = []
        for key in self.preds:
            arr = [{0: Fact.FALSE}]*(len(self.preds[key])*2)

            positive, value = raw_bin_dict[key]

            index = 2*(value-1) if positive else 2*value-1
            arr[index][0] = Fact.TRUE
            d.append(dict(zip(self.preds[key], arr)))
        res = {**d[0], **d[1], **d[2], **d[3]}
        return res

    class FOLCartpoleAgent():
        MAXLEN = 10_000
        MIN_REPLAY_SIZE = 1_000
        BATCH_SIZE = 64
        GAMMA = 0.9
        LR = 0.01

        def __init__(self, n_bin_args, n_nodes, limits):
            self.left_lnn = LNNCartpole(n_nodes, **n_bin_args)
            self.right_lnn = LNNCartpole(n_nodes, **n_bin_args)
            self.limits = limits
            self.bin_args = n_bin_args
            self.bin_sizes = {}
            for (key1, key2) in zip(self.limits, self.bin_args):
                self.bin_sizes[key1] = self.limits[key1][1]/self.bin_args[key2]

            self.replay_memory = collections.deque([], maxlen = self.MAXLEN)

        def env2fol(self, obs):
            assert obs.shape == (4,)
            obs = Observation(*obs)
            ret = {}
            for key in self.limits:
                val = getattr(obs, key)
                positive = (val >= 0)
                if positive:
                    val_bin = math.ceil(val/self.bin_sizes[key])
                if val/self.bin_sizes[key] - int(val/self.bin_sizes[key]) == 0:
                    val_bin += 1
                    val_bin = min(val_bin, self.bin_args[key])
                else:
                    val_bin = math.floor(val/self.bin_sizes[key])
                    val_bin = max(val_bin, -self.bin_args[key])

                ret[key] = (positive, abs(val_bin))
            return ret

        def remember(self, obs):
            '''
              obs: namedtuple given by (state, action, reward, next_state, done)
            '''
            self.replay_memory.append(obs)

        def optimize(self):
            if len(self.replay_memory) < self.MIN_REPLAY_SIZE:
                return

            transitions = [self.replay_memory[idx] for idx in np.random.permutation(len(self.replay_memory))[:self.MINIBATCH_SIZE]]
            batch = Transition(*zip(*transitions))

            batch.state

            action_batch = torch.tensor(batch.action, device = self.device, dtype = torch.int64)
            reward_batch = torch.tensor(batch.reward, device = self.device)

        def sample_random_action():
            



In [54]:
EPSILON_START = 1.0
EPSILON_END = 0.1
WARMUP_STEPS = 1_000
NUM_EPISODES = 10_000

n_bin_args = {
    "n_pos": 10,
    "n_vel": 10,
    "n_ang": 10,
    "n_angvel": 10
}

limits = {
    "Position": [-1.2, 1.2],
    "Velocity": [-2, 2],
    "Angle": [-0.2094395, 0.2094395],
    "AngVelocity": [-3, 3]
}


agent = FOLCartpoleAgent(n_bin_args, n_nodes = 10, limits = limits)
env = gym.make("CartPole-v1")

In [ ]:
def train():
    epsilon = EPSILON_START
    episode_num = 0
    for ep in tqdm.tqdm(range(NUM_EPISODES)):
        total_loss, step = 0,0
        state, info = env.reset()
        while True:
            if np.random.random() > epsilon:
                action = agent.sample_random_action()
            else:
                action = agent.get_action(state)
            curr_state = copy.deepcopy(env.current_state)
            next_state, reward, terminal, truncated, info = env.step(action)
            reward = reward if not terminal else 0
            next_state = torch.tensor(np.reshape(state_next, [1, observation_space]))
            dqn_solver.remember(Observation(state, action, reward, state_next, terminal))
            loss = agent.optimize()
            
            state = next_state

            if loss is not None:
                episode_loss += loss
                step += 1
            
            if terminal or truncated:
                print("Run: " + str(episode) + ", score: " + str(step) + ", episode_loss: " + str(total_loss/step))
                episode_runs.append(step)
                episode_losses.append(episode_loss/step)
                epsilon = epsilon * epsilon_decay
                epsilon = min(epsilon, epsilon_end)
                break

In [46]:
def train():
		epsilon = EPS_START
		episode_num = 0
		step = 0
		for ep in tqdm.tqdm(range(NUM_EPISODES)):
				total_loss, step = 0, 0
				for query in database.queries:
						env.reset(query)
						while True:
								if env.current_length == MAX_REGEX_LENGTH-1:
									action = vocab.word_dict["$"]
								else:
											if np.random.random() > epsilon:
													action = agent.get_action(query, env.current_state, env.current_length)
											else:
													action = env.sample_random_action()

								curr_state = copy.deepcopy(env.current_state)

								next_state, reward, length, done = env.step(action)
								if next_state is not None:
										next_state = torch.tensor(next_state)
										# if length > 1:
										#     temp_reward = env.hypothesis(curr_state, length)
										#     agent.update_replay_memory(Transition(
										#         torch.tensor(query), torch.tensor(curr_state), torch.tensor(length+1), torch.tensor(vocab.word_dict["$"]),
										#         torch.tensor([-1]*10), torch.tensor(temp_reward), torch.tensor(True)))
								else:
										next_state = torch.tensor([-1]*10)
								agent.update_replay_memory(Transition(
										torch.tensor(query), torch.tensor(curr_state), torch.tensor(length), torch.tensor(action),
										next_state, torch.tensor(reward), torch.tensor(done)))
								loss = agent.optimize()
								if loss is not None:
										total_loss += loss
										step += 1
								# if step > 0 and step % 100 == 0:
								# 		print("step {:d} | loss {:.3f} | lr {:.5f}".format(
								# 				step, total_loss/step, agent.optimizer.param_groups[0]['lr']))
								# 		losses.append(total_loss)
								# 		steps.append(step)
								loss = agent.optimize(only_success=True)
								if loss is not None:
										total_loss += loss
										step += 1
								if done:
										break

						episode_num += 1
						if len(agent.replay_memory) > agent.MIN_REPLAY_MEMORY_SIZE and episode_num % agent.update_target_every == 0:
								print("model updated")
								agent.target_model.load_state_dict(agent.policy_model.state_dict())
				if step > 0:
					print("ep {:d} | loss {:.3f}".format(ep, total_loss/step))
				# Decay epsilon
				if len(agent.replay_memory) > agent.MIN_REPLAY_MEMORY_SIZE and epsilon > EPS_END:
						epsilon *= EPS_DECAY
						epsilon = max(EPS_END, epsilon)

				if len(agent.replay_memory) > agent.MIN_REPLAY_MEMORY_SIZE:
						if ep % 10 == 0:
								prec, rec, error= test(verbose = True)
						else:
								prec, rec, error= test(verbose = False)
    #torch.save(agent.policy_model.state_dict(), "model")